In [1]:
# #This is how to get client-side rendered page
# #You can get all information you want but it will slower than the first approach because it needs to wait until js finish its job
# from requests_html import HTMLSession, AsyncHTMLSession

# import asyncio
# if asyncio.get_event_loop().is_running():
#     import nest_asyncio
#     nest_asyncio.apply()

# url = "https://search-merchant.xn--42caj4e6bk1f5b1j.com/result?searchQuery=&provinceCode=83&categoryId=50001%2C10001%2C10007"
# asession = AsyncHTMLSession()
# r = await asession.get(url)
# await r.html.arender()
# resp=r.html.raw_html


In [2]:
# resp

In [3]:
!pip install webdriver-manager

In [4]:
!pip install selenium-wire

In [5]:
from seleniumwire import webdriver

In [6]:


from selenium.webdriver.chrome.options import Options
# from selenium import webdriver

chrome_options = Options()
chrome_options.add_argument("--headless")

from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Tk\.wdm\drivers\chromedriver\win32\86.0.4240.22]


In [7]:
driver

<seleniumwire.webdriver.browser.Chrome (session="6487385d9646655054634652f49e1df8")>

In [9]:
import json

# format: { province_code: number of shops in that province}
province_code = json.load(open("province_code.json"))

In [14]:
import json

def get_shops_from_requests(driver):
    shops = []
    for request in driver.requests:
        if request.path=="/api/shop/search":
            items = json.loads(request.response.body)
            shops = shops + items["data"]
    return shops


In [57]:
from selenium.common.exceptions import JavascriptException
import time
import random

province_names = {}
for k in province_code.keys():
    k = "85"
    url = f"https://search-merchant.xn--42caj4e6bk1f5b1j.com/result?searchQuery=&provinceCode={k}&categoryId=all"
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    driver.get(url)
    print("SLEEP 5 to settle down")
    time.sleep(5)
    
    shops = get_shops_from_requests(driver)
    print(k, shops[0]["addressProvinceName"], province_code[k])
    
    province_names[k] = shops[0]["addressProvinceName"]
    # It will click next until it gets all items; each click will get around 18 items
    n = int((province_code[k]//18) * 1.5)
    for i in range(n):
        try:
            driver.execute_script('document.querySelector(".items-start>.flex-1 button").click()')
            if (i%50==0):
                items = driver.find_elements_by_css_selector('.items-start>.flex-1 .grid>a')
                print(f"{i}/{n}", len(items))

            time.sleep(1+random.random())
        except JavascriptException as e:
            print(e)
            break
    
    print("SLEEP 5 to settle down again")
    time.sleep(5)
    
    shops = get_shops_from_requests(driver)
    o = json.dumps(shops, indent=4)
    fo = open(f"shop_{k}.json", "w")
    fo.write(o)
    fo.close()
    
    print(k, len(shops), len(shops)*100.0/province_code[k])
    print("----------------")
    
    break

o = json.dumps(province_names, indent=4)
fo = open(f"province_names.json", "w")
fo.write(o)
fo.close()



[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\Tk\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


 
SLEEP 5 to settle down
85 ระนอง 626
0/51 18
Message: javascript error: Cannot read property 'click' of null
  (Session info: headless chrome=86.0.4240.111)

SLEEP 5 to settle down again
85 628 100.31948881789137
----------------


In [ ]:
# get_shops_from_requests(driver)

In [96]:
# items = driver.find_elements_by_css_selector('.items-start>.flex-1 .grid>a')
# len(items)

683

Message: javascript error: Cannot read property 'click' of null
  (Session info: headless chrome=86.0.4240.111)

